<a href="https://colab.research.google.com/github/haseebahmer/Brain_Tumor_Colab/blob/main/Brain_Tumor_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class SimpleBrainTumorDataset(Dataset):
    def __init__(self, yes_dir, no_dir, transform=None):
        self.transform = transform
        self.images = []
        self.labels = []
        self._load_images(yes_dir, label=1)  # 1 for tumor present
        self._load_images(no_dir, label=0)   # 0 for no tumor

    def _load_images(self, directory, label):
        for filename in os.listdir(directory):
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                self.images.append(os.path.join(directory, filename))
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
yes_dir = '/content/drive/MyDrive/yes'  # Update these paths
no_dir = '/content/drive/MyDrive/no'    # Update these paths


In [ ]:
display(yes_dir)

'/content/drive/MyDrive/yes'

In [ ]:
dataset = SimpleBrainTumorDataset(yes_dir=yes_dir, no_dir=no_dir, transform=transform)


In [ ]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
class SimpleANN(nn.Module):
    def __init__(self):
        super(SimpleANN, self).__init__()
        self.fc1 = nn.Linear(224*224*3, 512)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 128)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.flatten(x)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        return self.fc3(x)

# Function to create VGG16 model
def create_vgg_model():
    model = models.vgg16(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    model.classifier[6] = nn.Linear(model.classifier[6].in_features, 2)
    return model

# Function to create ResNet18 model
def create_resnet_model():
    model = models.resnet18(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    model.fc = nn.Linear(model.fc.in_features, 2)
    return model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models = {
    "SimpleANN": SimpleANN().to(device),
    "VGG16": create_vgg_model().to(device),
    "ResNet18": create_resnet_model().to(device)
}


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
 

In [ ]:
def train_model(model, loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in loader:
            images, labels = images, labels  # Ensure tensors are on the CPU

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(loader):.4f}')


In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
for name, model in models.items():  # Correctly iterate over the dictionary
    print(f"Training {name}...")
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train_model(model, train_loader, criterion, optimizer)


Training SimpleANN...
Epoch 1/10, Loss: 34.8925
Epoch 2/10, Loss: 24.0062
Epoch 3/10, Loss: 17.4247
Epoch 4/10, Loss: 11.2268
Epoch 5/10, Loss: 10.8045
Epoch 6/10, Loss: 14.1399
Epoch 7/10, Loss: 7.6412
Epoch 8/10, Loss: 5.9812
Epoch 9/10, Loss: 5.2215
Epoch 10/10, Loss: 2.9772
Training VGG16...
Epoch 1/10, Loss: 0.6543
Epoch 2/10, Loss: 0.5257
Epoch 3/10, Loss: 0.4096
Epoch 4/10, Loss: 0.4954
Epoch 5/10, Loss: 0.3787
Epoch 6/10, Loss: 0.3544
Epoch 7/10, Loss: 0.3253
Epoch 8/10, Loss: 0.3147
Epoch 9/10, Loss: 0.3054
Epoch 10/10, Loss: 0.2823
Training ResNet18...
Epoch 1/10, Loss: 0.9760
Epoch 2/10, Loss: 0.7493
Epoch 3/10, Loss: 0.6983
Epoch 4/10, Loss: 0.6043
Epoch 5/10, Loss: 0.5425
Epoch 6/10, Loss: 0.5416
Epoch 7/10, Loss: 0.4849
Epoch 8/10, Loss: 0.4163
Epoch 9/10, Loss: 0.4003
Epoch 10/10, Loss: 0.3702


In [42]:
def evaluate_model(model, loader):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images, labels  # Ensure tensors are on the CPU
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.numpy())
            all_preds.extend(preds.numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    cm = confusion_matrix(all_labels, all_preds)

    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'Confusion Matrix:\n{cm}')

# Evaluating each model on the test data
for name, model in models.items():
    print(f"Evaluating {name} on test data...")
    evaluate_model(model, test_loader)

Evaluating SimpleANN on test data...
Accuracy: 0.6944
Precision: 0.6207
Recall: 1.0000
Confusion Matrix:
[[ 7 11]
 [ 0 18]]
Evaluating VGG16 on test data...
Accuracy: 0.8889
Precision: 1.0000
Recall: 0.7778
Confusion Matrix:
[[18  0]
 [ 4 14]]
Evaluating ResNet18 on test data...
Accuracy: 0.8611
Precision: 0.8095
Recall: 0.9444
Confusion Matrix:
[[14  4]
 [ 1 17]]


In [43]:
best_model_name = "VGG16"
torch.save(models[best_model_name].state_dict(), '/content/drive/MyDrive/best_model.pth')